# BioBuilder API Demo

In [1]:
import sys

sys.path.insert(0, '..')

import os
import requests

caravagna_model_filepath = '../biosimulator_processes/model_files/Caravagna2010.xml'
print(os.path.exists(caravagna_model_filepath))

tumor_control_biomodel_id = 'BIOMD0000000749'
requests.get('https://www.ebi.ac.uk/biomodels/BIOMD0000000749', headers={'accept': 'json'})

True


<Response [200]>

#### **_Experiment 1_**:  Here we cross a boundary in the stack that is a biological simulation. We go from model configuration, to experiment. Thus, this tooling sits at that level: both experiment specification AND experiment execution. It's not just a way to specify an experiment, but it is also a way to run it, given the many knobs and buttons that you can use predefine and customize the way the actual model is being solved. Our users are seeking to be involved in an experiment as a "stack". We should make a bigger distinction between terms like "model".

In [2]:
from biosimulator_processes.biosimulator_builder import BuildPrompter
from biosimulator_processes.data_model import TimeCourseProcess, TimeCourseModel

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.


In [3]:
# 1. define a model for the process composition. In this case, just one model to be re-used as configuration for the processes we create:

tc_model = TimeCourseModel(model_source=tumor_control_biomodel_id)

In [4]:
# >> The model is viewable as a dataclass...

tc_model

TimeCourseModel(model_source='BIOMD0000000749', model_id='model_from_BIOMD0000000749', model_name='model_from_BIOMD0000000749', model_language='sbml', model_changes=None, model_units=None)

In [5]:
# >> ...or a dict:

tc_model.dump()

{'model_source': 'BIOMD0000000749',
 'model_id': 'model_from_BIOMD0000000749',
 'model_name': 'model_from_BIOMD0000000749',
 'model_language': 'sbml',
 'model_changes': None,
 'model_units': None}

In [6]:
# 2. instantiate the prompter:

prompter = BuildPrompter()

In [ ]:
# 3. add process(es) to the bigraph with the Time Course model instance we created above. For now, just one process will be added.

prompter.add_single_process(config=tc_model)

#### **_Experiment 2_**: Load an SBML model from a specified model filepath and add Model changes to the composite before adding it to the bigraph.

In [ ]:
from biosimulator_processes.data_model import TimeCourseModelChanges, ModelChanges, ModelChange